In [4]:
#%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy.optimize import curve_fit # Az illesztéshez használt függvény
import math
import scipy
import statistics

# Diffusion coefficient from time difference (errorcfn)

In [5]:
def gaussian_diffusion(x,c,Dt):
    return c*np.exp(-x**2/(4*Dt))

def errorc_fn(x,c,Dt):
    return c*scipy.special.erfc(x/(2*math.sqrt(Dt)))

In [6]:
root_path = r'C:\Users\CFY\Documents\MFA\Measurements\CTAX\Diffusion\New_2022\IgG\20220310\chip3\x451'
path_format = "{}\Values{}.csv"
channel_length = 800
line_length_px_y = 549-32
material='IgG'
time=0.5
profile_position=446

number_of_points_deleted=5
line_length_px=line_length_px_y-number_of_points_deleted
maximum_intensity=0

for i in range(14202,14207):
    data=np.genfromtxt('{}\Values{}.csv'.format(root_path,i), delimiter=',', skip_header=1)
    max_in_array= np.amax(data[:,1])
    if max_in_array > maximum_intensity:
        maximum_intensity = max_in_array

In [8]:
myfile = open('{}\D_x{}_{}_{}_min_vege.txt'.format(root_path,profile_position,material,time), 'w')
myfile.writelines("Time difference between profiles: " + str(time) + " minutes" + "\n \n")
myfile.writelines("D (um2/s): " + "\t" + "Errors:" "\n")

diff_coeffs_all=[]

for i in range(14202,14207,int(time*2)):
    data_ok=np.genfromtxt('{}\Values{}.csv'.format(root_path,i), delimiter=',',skip_header=1)
    data2_ok=np.genfromtxt('{}\Values{}.csv'.format(root_path,i+int(time*2)), delimiter=',',skip_header=1)
    
    
    for i in range(number_of_points_deleted):
        data_ok=np.delete(data_ok,0,0)
        data2_ok=np.delete(data2_ok,0,0)
    
    data_ok[:,1] = data_ok[:,1]/maximum_intensity
    data2_ok[:,1] = data2_ok[:,1]/maximum_intensity
    
    shift_1=round(np.amin(data_ok[1:,]),3)+0.01 
    shift_2=round(np.amin(data2_ok[1:,]),3)+0.01
    
    data_ok[:,0] = data_ok[:,0]*(800/(line_length_px))*10**-6
    data2_ok[:,0] = data2_ok[:,0]*(800/(line_length_px))*10**-6

    data_ok[:,1] = data_ok[:,1]-shift_1
    data2_ok[:,1] = data2_ok[:,1]-shift_2
    
    #plt.plot(data_ok[:,0],data_ok[:,1],data2_ok[:,0],data2_ok[:,1])
    
    initial_guess = [1,0.00000000001]
    popt, pcov = curve_fit(errorc_fn, data_ok[:,0], data_ok[:,1],p0=initial_guess)

    D1=popt[1]
    perr1 = np.sqrt(np.diag(pcov))

    initial_guess = [1,0.00000000001]
    popt, pcov = curve_fit(errorc_fn, data2_ok[:,0], data2_ok[:,1],p0=initial_guess)

    D2=popt[1]
    perr2 = np.sqrt(np.diag(pcov)) 

    D=(D2-D1)/(60*time)

    if perr1[1]/D1 > perr2[1]/D2:
        p_err=perr1[1]/D1*100
    else:
        p_err=perr2[1]/D2*100
        
    D=round(D*10**12,3)
    diff_coeffs_all.append(D)
        
    myfile.writelines(str(D) + "\t" + "\t" + str(round(p_err,3)) + "\n")
    
myfile.writelines("\n")    
myfile.writelines("Average D from values: " + str(round(statistics.mean(diff_coeffs_all),3)))
myfile.writelines("\n")  

myfile.close() 
#print(statistics.mean(diff_coeffs_all))


38.003
